# E11 - 멋진 작사가 만들기

In [1]:
# 데이터 불러오기

import glob
import os

import re
import numpy as np
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel//lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['The Cat in the Hat', 'By Dr. Seuss', 'The sun did not shine.']


In [2]:
# 전처리 함수 구현

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 문장 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    # 패턴의 특수문자[?.!,¿]를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r'[" "]+', " ", sentence)
    # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'   # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다.
    
    return sentence

In [3]:
# raw_corpus를 전처리 하기
 
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
            
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> the cat in the hat <end>',
 '<start> by dr . seuss <end>',
 '<start> the sun did not shine . <end>',
 '<start> it was too wet to play . <end>',
 '<start> so we sat in the house <end>',
 '<start> all that cold cold wet day . <end>',
 '<start> i sat there with sally . <end>',
 '<start> we sat there we two . <end>',
 '<start> and i said how i wish <end>',
 '<start> we had something to do ! <end>']

In [4]:
# 문장을 토큰화하여 tensor로 만드는 함수 구현

def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,    # 전체 단어의 개수 
        filters=' ',       # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    
    corpus = [x for x in corpus if len(x.split()) <= 15]
    
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.
    
    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding 메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
           
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2    6  824 ...    0    0    0]
 [   2  115 2805 ...    0    0    0]
 [   2    6  286 ...    0    0    0]
 ...
 [   2 2412   13 ...    0    0    0]
 [   2  716   27 ...    0    0    0]
 [   2  716   27 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f35e35b6190>


In [5]:
print(tensor[3])

[  2  11  53 102 898  10 210  22   3   0   0   0   0   0   0]


In [6]:
from sklearn.model_selection import train_test_split

src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=2020)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124981, 14)
Target Train: (124981, 14)


In [7]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [8]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 1024   # 워드 벡터의 차원수, 즉 단어가 추상적으로 표현되는 크기 (예, 2일 경우 차갑다:[0.0, 1.0]
hidden_size = 2048     # LSTM layer의 hidden state의 차원수
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [9]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 7.86827877e-05, -2.03900763e-05, -6.95526018e-04, ...,
         -6.81765960e-06, -2.31695831e-05,  2.53590115e-04],
        [-2.83458503e-04,  4.05584797e-05, -1.48646138e-03, ...,
          1.77562397e-04,  1.65452686e-04,  2.32002552e-04],
        [-1.34194590e-04, -5.85424596e-06, -1.83883507e-03, ...,
          8.88134469e-04,  5.26629738e-04, -5.40162400e-05],
        ...,
        [-1.73615350e-04,  3.84138722e-04, -1.07307616e-03, ...,
         -1.14428950e-03, -7.58962648e-04, -9.01418505e-04],
        [-1.84503340e-04, -6.67712229e-05, -7.92522740e-04, ...,
         -1.05119171e-03, -5.56493411e-04, -8.91970529e-04],
        [-2.56917177e-04, -3.09066236e-05, -8.20186338e-04, ...,
         -8.17465771e-04, -1.86844889e-04, -3.03546432e-04]],

       [[ 7.86827877e-05, -2.03900763e-05, -6.95526018e-04, ...,
         -6.81765960e-06, -2.31695831e-05,  2.53590115e-04],
        [-2.85995939e-05,  2.30611055e-04, -8

In [10]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  12289024  
_________________________________________________________________
lstm (LSTM)                  multiple                  25174016  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense (Dense)                multiple                  24590049  
Total params: 95,615,713
Trainable params: 95,615,713
Non-trainable params: 0
_________________________________________________________________


In [11]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10, validation_data = (enc_val, dec_val))

Epoch 1/10
488/488 [==============================] - 237s 485ms/step - loss: 3.2308 - val_loss: 2.8617
Epoch 2/10
488/488 [==============================] - 234s 479ms/step - loss: 2.6790 - val_loss: 2.6024
Epoch 3/10
488/488 [==============================] - 238s 487ms/step - loss: 2.3477 - val_loss: 2.4208
Epoch 4/10
488/488 [==============================] - 235s 482ms/step - loss: 2.0239 - val_loss: 2.2882
Epoch 5/10
488/488 [==============================] - 232s 476ms/step - loss: 1.7220 - val_loss: 2.1953
Epoch 6/10
488/488 [==============================] - 232s 476ms/step - loss: 1.4626 - val_loss: 2.1415
Epoch 7/10
488/488 [==============================] - 232s 476ms/step - loss: 1.2587 - val_loss: 2.1200
Epoch 8/10
488/488 [==============================] - 232s 476ms/step - loss: 1.1162 - val_loss: 2.1306
Epoch 9/10
488/488 [==============================] - 232s 476ms/step - loss: 1.0316 - val_loss: 2.1469
Epoch 10/10
488/488 [==============================] - 232s 476m

In [12]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다.
        # 우리 모델이 예측한 마지막 단어([:, -1])가 바로 새롭게 생성한 단어가 됩니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <END>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [19]:
generate_text(model, tokenizer, init_sentence="<start> i know", max_len=20)

'<start> i know you re unhappy , <end> '

지나치게 긴 문장은 다른 데이터들이 과도한 padding을 갖게 된다고 토큰 갯수가 15개 넘어가면 삭제하라고   
권고하였는데 과도한 padding이 결과에 어떠한 영향을 미칠까?   

우선 토큰의 갯수에 따라  
embedding_size = 256, hidden_size = 1024  
14개 이하일 때   
loss: 2.2110, val_loss: 2.5187  
i love you, i m better   

32개 이하일 때   
loss: 1.1226, val_loss: 1.243   

64개 이하일 때
loss: 0.4401, val_loss: 0.5726  
i know you re smiling out of time/ i love you, i love you, .. 반복, 후렴 구간인가?

토큰 갯수가 증가할수록, padding이 증가할수록 오히려 loss가 낮아졌다.   
과도하다고 해서 불리한 결과가 나올 줄 알았는데 원인이 뭘까?   

embedding_size = 1024, hidden_size = 2048 조정   
14개 이하일 때
loss: 0.9889, val_loss: 2.1637   
i know you re unhappy/ i love you   
val_loss 0.35정도 하락, 40분 가까이 걸린 거에 비하면 아주 조금 하락   
32개, 64개이하는 시간이 얼마나 걸릴 지 엄두가 안 남.   
아마도 두 배이상 걸릴 것 같으나 수치 하락은 미미할 것 같음.   

사전에 있는 단어들로 조합하는 것이지만 새로운 것을 만들어낼 수 있다는 것이 재밌음.